## Create Keras model using functional API

### Neural Network Embeddings
An embedding is a mapping of a discrete — categorical — variable to a vector of continuous numbers. In the context of neural networks, embeddings are low-dimensional, learned continuous vector representations of discrete variables. Neural network embeddings are useful because 
- they can reduce the dimensionality of categorical variables and 
- reduce sparsity in input activations, which helps reduce overfitting 

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

%run utils.ipynb
df = testimon_fraud()

In [ ]:
from keras.layers import *
from keras.models import *

# Create first layer for the categorical variable

num_types = len(df['type'].unique()) 
type_embedding_dim = 3 # arbitrary choice

type_in = Input(shape=(1,))
type_embedding = Embedding(num_types,type_embedding_dim,input_length=1)(type_in)
type_out = Reshape(target_shape=(type_embedding_dim,))(type_embedding)

type_model = Model(type_in,type_out)

inputs = []
outputs = []
inputs.append(type_in)
outputs.append(type_out)



In [ ]:
# Create another input for all other variables (non-categorial ones)

other_cols = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
df['nameOrig'] = df.nameOrig.astype('category')
df['nameDest'] = df.nameDest.astype('category')
print(other_cols)
num_rest = len(other_cols)

rest_in = Input(shape = (num_rest,)) 
rest_out = Dense(16)(rest_in)

rest_model = Model(rest_in,rest_out)

inputs.append(rest_in)
outputs.append(rest_out)

In [ ]:
# Build the model
concatenated = Concatenate()(outputs)
x = Dense(16)(concatenated)
x = Activation('sigmoid')(x)
x = Dense(1)(concatenated)
model_out = Activation('sigmoid')(x)

merged_model = Model(inputs, model_out)
merged_model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
types = df.type.astype('category').cat.codes
rest = df[other_cols]
target = df['isFraud']
history = merged_model.fit([types.values,rest.values],target.values, epochs = 1, batch_size = 128)
